# TensorFlow ELM example with Normal Equation Choleski-based Matrix Solver 

In [1]:
import keras 
from keras.datasets import mnist
import os
import tensorflow as tf

/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
mnist_data = mnist.load_data(os.path.join(os.getcwd(), 'mnist_dataset.csv'))

In [3]:
train_set, test_set = mnist_data
x_train, y_train = train_set
x_test, y_test = test_set

In [4]:
y_train = keras.utils.to_categorical(y_train, num_classes=10)
y_test = keras .utils.to_categorical(y_test, num_classes=10)

In [5]:
x_train = x_train.reshape(-1,28*28)
x_test=x_test.reshape(-1,28*28)
x_train.shape

(60000, 784)

## define ELM core computational TF graph 

In [9]:
with tf.name_scope("elm"): 
    
    with tf.name_scope("input"):
        input = tf.placeholder(tf.float32, shape=[None, 28*28], name="input")
        y = tf.placeholder(tf.float32, shape=[None, 10], name="input")
           
        
    with tf.name_scope("hidden_layer_1"):
        weights = tf.Variable(tf.random_normal(shape=[28*28,1024], stddev=1), trainable=False)
        biases = tf.Variable(tf.random_normal(shape=[1024], stddev=1),trainable=False)
        
        pre_activations = tf.matmul(input, weights) + biases
        
        activations = tf.sigmoid(pre_activations)
        
    

## Training and Testing of ELM 

**note how training is done via batching. 
After the activation matrix has been obtained (conc in the code) the solver is called.**

**Testing is made in one-pass thereafter**
** note how it is necessary to specify a different graph part for testing and prediction**

In [21]:
# get activations via batches 
with tf.Session() as sess:
    tf.initialize_all_variables().run()
    print("initialized")
    batch_size = 10000 
    
    num_batches = x_train.shape[0]//batch_size
    
    
    for i in range(num_batches):
        
        X_train = x_train[i*batch_size:((i+1)*batch_size)].astype('float32')
        Y_train = y_train[i*batch_size:((i+1)*batch_size)].astype('float32')
        print(X_train.shape)
        if i == 0: 
            act_prev = sess.run(activations,feed_dict={input:X_train, y:Y_train})
        else:
            act = sess.run(activations,feed_dict={input:X_train, y:Y_train})
            conc=tf.concat((act_prev,act),axis=0)
            act_prev=conc
    
    with tf.name_scope("optimizer"):
        optimum = tf.matrix_solve_ls(conc, y, 0,fast=True) 
        # the second is L2 reg parameter optimal is 100 to 1000

    with tf.name_scope("output_layer"):
        output = tf.matmul(conc,optimum)
        
    
    with tf.name_scope("accuracy"):
        correct_prediction = tf.equal(tf.argmax(output, 1), tf.argmax(y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        
    with tf.name_scope("test"):
        opt = tf.placeholder(tf.float32, shape=[1024, 10], name="input_for_test")
        output_test = tf.matmul(activations,opt)
        test_correct_prediction = tf.equal(tf.argmax(output_test, 1), tf.argmax(y, 1))
        accuracy_test = tf.reduce_mean(tf.cast(test_correct_prediction, tf.float32))
    
    
    print("initialized")

    
    acc_train,weights=sess.run([accuracy,optimum],feed_dict={input:x_train, y:y_train})
    acc_test=sess.run([accuracy_test],feed_dict={input:x_test, y:y_test, opt:weights})
        

initialized
(10000, 784)
(10000, 784)
(10000, 784)
(10000, 784)
(10000, 784)
(10000, 784)
initialized


In [24]:
print(acc_test)
print(acc_train)

[0.9216]
0.9213


[0.0992]